### Setup domain:

notebooks/Experimental/Ishan/AA/setup-data.ipynb

In [1]:
import syft as sy
from tqdm import tqdm
import time

In [2]:
domain_addresses = [8081, 8082]  # replace with URLs
username = "info@openmined.org"  # "sam@stargate.net"
pw = "changethis"

In [4]:
from syft import nn

def initialize_model() -> nn.Model:
    model = nn.Model()

    # Layer 1
    model.add(nn.Convolution(nb_filter=32, filter_size=3, padding=2, input_shape=input_shape))
    model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    model.add(nn.MaxPool(pool_size=2, stride=2))

    # Layer 2
    model.add(nn.Convolution(nb_filter=64, filter_size=3, padding=2))
    model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    model.add(nn.MaxPool(pool_size=2, stride=2))

    # # Layer 3
    # model.add(nn.Convolution(nb_filter=128, filter_size=3, padding=2))
    # model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    # model.add(nn.MaxPool(pool_size=2, stride=2))

    # # Layer 4
    # model.add(nn.Convolution(nb_filter=256, filter_size=3, padding=2))
    # model.add(nn.BatchNorm(activation=nn.leaky_ReLU()))
    # model.add(nn.MaxPool(pool_size=2, stride=2))

    # Layer 5
    model.add(nn.AvgPool(2))

    # Layer 6
    model.add(nn.Flatten())

    # Layer 7
    model.add(nn.Linear(2, 512))

    model.initialize_weights()
    return model

In [6]:
# TRAINING PARAMS
n_epochs = 1
batch_size = 2
input_shape = (2,3,50,50)
model = initialize_model()

for address in tqdm(domain_addresses):
    domain = sy.login(email=username, password=pw, port=address)  # change to URL when running on 100
    
    if len(domain.datasets) == 0:
        print(f"Error on domain = {domain.name} with address = {address}")
    
    X_train = domain.datasets[-1]["train_images"][:2]
    y_train = domain.datasets[-1]["train_labels"][:2]
    input_shape = X_train.public_shape
    
    # Do a single epoch
    model_ptr = model.send(domain,send_to_blob_storage=False)
    run_status = model_ptr.step(X_train, y_train)
    while(not run_status.exists):
        time.sleep(10)
    
    # Publish Model Weights
    published_obj = model_ptr.publish(sigma=1e4)  # PB spent with 1e3 = 690k, thus 1e4 sigma -> 6.9k which is within PB limit of 9999
    while(not published_obj.exists):
        time.sleep(2)
    new_weights = published_obj.get_copy()
    
    # Update weights and move onto next domain node
    model = initialize_model()  # replacing weights into existing model causes a NotImplementedError
    model.replace_weightsace_weights(new_weights)    

  0%|                                                                                                                           | 0/2 [00:00<?, ?it/s]


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


 50%|█████████████████████████████████████████████████████████                                                         | 1/2 [04:54<04:54, 294.51s/it]

['Convolution0', 'BatchNorm1', 'MaxPool2', 'Convolution3', 'BatchNorm4', 'MaxPool5', 'AvgPool6', 'Flatten7', 'Linear8']

Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into italy... done!


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [09:46<00:00, 293.50s/it]

['Convolution0', 'BatchNorm1', 'MaxPool2', 'Convolution3', 'BatchNorm4', 'MaxPool5', 'AvgPool6', 'Flatten7', 'Linear8']


In [7]:
print(new_weights)

{'Convolution0': [array([[[[ 1.97288876e+04,  1.86930355e+04, -7.16844789e+03],
         [ 1.29362669e+03, -4.52495460e+03,  1.01005585e+04],
         [-1.58713212e+03, -1.77300837e+04, -4.50892252e+03]],

        [[ 1.72364315e+03, -5.35957385e+03, -3.26207999e+03],
         [ 7.84565485e+03, -6.28506568e+03, -5.21041707e+03],
         [-8.52814147e+03, -3.92548053e+03, -1.53455023e+04]],

        [[-3.06480911e+03,  2.47705880e+04,  1.31039713e+04],
         [-4.27667180e+03, -1.39566308e+04, -1.97522495e+04],
         [-2.22690679e+04,  6.26265273e+02, -1.99901310e+03]]],


       [[[-1.77355330e+04,  2.37193259e+03, -1.08601475e+04],
         [ 1.16895240e+04, -8.44181481e+03, -6.23876701e+02],
         [-5.02252602e+03,  4.33359112e+03, -5.67970628e+03]],

        [[ 8.26878197e+03,  9.33262408e+03, -2.50923600e+03],
         [ 1.27091348e+04, -1.57992520e+04, -6.32567907e+03],
         [ 6.66014936e+03, -5.53908483e+03,  2.00668468e+04]],

        [[ 5.59484288e+02,  1.52393604e+